In [1]:
!pip install cirq

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 1.2MB 47.3MB/s 
     |████████████████████████████████| 5.3MB 35.4MB/s 
     |████████████████████████████████| 1.8MB 29.4MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556408 sha256=3f0f6bb9e78ac09bf8f86ac6f7e20b27b4042418c81c816c176fbdad8884f444
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0
  Found existing installation: sympy 1.1.1
    Uninstalling sympy-1.1.1:
      Successfully uninstalled sympy-1.1.1
  Found existing installation: networkx 2.4
    Uninstalling networkx-2.4:
      Successfully uninstalled networkx-2.4


In [0]:
import cirq
import random

In [0]:
# Number of input qubits
qubitNo=8
inputQubit=[cirq.GridQubit(i,0) for i in range(qubitNo)]

# Output qubit
outputQubit=cirq.GridQubit(qubitNo,0)

# String S such that f(x)=x.s 
factorString=[random.randint(0,1) for _ in range(qubitNo)]


In [0]:
# define Oracle U_f
def makeOracle(inputQubit, outputQubit, factorString):
  for qubit, bit in zip(inputQubit,factorString):
    if bit:
      yield cirq.CNOT(qubit, outputQubit)

In [0]:
# define the whole BV algorithm circuit
def makeBVcircuit(inputQubit, outputQubit, Oracle):
  BV=cirq.Circuit()
  BV.append([
  cirq.X(outputQubit),
  cirq.H(outputQubit),
  cirq.H.on_each(*inputQubit)             
  ])

  BV.append(Oracle)

  BV.append([
  cirq.H.on_each(*inputQubit),
  cirq.measure(*inputQubit, key='result')          
  ])

  return BV  

In [51]:
#make one example of Oracle and BVcircuit
Oracle=makeOracle(inputQubit, outputQubit, factorString)

BVcircuit=makeBVcircuit(inputQubit, outputQubit, Oracle)
print('BVCircuit')
print(BVcircuit)


BVCircuit
(0, 0): ───H───H───────────────M('result')───
                               │
(1, 0): ───H───────@───H───────M─────────────
                   │           │
(2, 0): ───H───H───┼───────────M─────────────
                   │           │
(3, 0): ───H───H───┼───────────M─────────────
                   │           │
(4, 0): ───H───H───┼───────────M─────────────
                   │           │
(5, 0): ───H───────┼───@───H───M─────────────
                   │   │       │
(6, 0): ───H───H───┼───┼───────M─────────────
                   │   │       │
(7, 0): ───H───H───┼───┼───────M─────────────
                   │   │
(8, 0): ───X───H───X───X─────────────────────


In [52]:
#measurement 
simulator=cirq.Simulator()
result=simulator.run(BVcircuit)

#comparison measurement result with factorString
print(result)
print('factorString=',factorString)

result=0, 1, 0, 0, 0, 1, 0, 0
factorString= [0, 1, 0, 0, 0, 1, 0, 0]
